
# Assignment 1 - Nash Equilibria, $\varepsilon$-NE, and Structured Analyses


<div style="border: 3px solid #222; padding: 16px; border-radius: 10px; background-color: #1c1f26; font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif; color: #e0e0e0;">
  <div style="display: flex; align-items: center; gap: 8px; margin-top: 12px;">
    <span style="font-size: 24px; color: #ff5555;">&#128274;</span>
    <span style="font-size: 16px;"><strong>Project:</strong> Homeworks</span>
  </div>
  <div style="display: flex; align-items: center; gap: 8px; margin-top: 8px;">
    <span style="font-size: 20px; color: #ff5555;">&#128218;</span>
    <span style="font-size: 16px;"><strong>Course:</strong> Game Theory</span>
  </div>
  <div style="margin-top: 12px; font-size: 14px;">
    <span style="font-size: 18px; color: #6e8192;">&#128100;</span>
    <span style="font-weight: bold;"><strong>Authors:</strong></span> Tamás Takács (PhD Student, Department of Artificial Intelligence, Eötvös Loránd University), László Gulyás (Associate Professor, Department of Artificial Intelligence, Eötvös Loránd University)
  </div>
</div>
<hr style="border: none; border-top: 2px solid #444;">
<br>

<img src="https://www.investopedia.com/thmb/z5ME0kPtBqbZ_HSfVa5y5hzrC6A=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/Nash-Equilibrium-FINAL-af47b1e9e53743759abc7b35997da583.jpg" alt="1" border="5">

This notebook contains the required task for the **first assignment** of the **Game Theory (GTEG)** course. Read the task description carefully and **fill in the empty code cells**.

# **Task Description**

In this homework exercise, you will practice concepts related to **Nash Equilibria** ($\text{NE}$), focusing on the following areas:

***

1.  **Pure Nash Equilibria (PNE)** (10 points)
2.  **Pure Nash Equilibria (PNE) using `nashpy`** (5 points)
3.  **$\varepsilon$-NE gap calculator for mixed profiles** (10 points)
4.  **Statistical Analysis of Random Games** (10 points)
5.  **Identifying Strictly Dominated Strategies** (10 points)
5.  **Axelrod Tournament Simulation** (5 points)

***

**Total Points Possible:** 50



## **Assignment Requirements and Guidelines**

Please adhere to the following rules to ensure your submission is graded correctly and fairly.

---

### **General Rules**

* You **must use the libraries** and tools specifically **mentioned** within the notebook to complete the exercises.
* Full points are awarded only when your solutions **pass all corresponding automated tests.**
* Copying code from any external source or another student will automatically result in a **score of 0** for the entire assignment and may lead to further academic penalties. These homeworks are for your personal practice.

### **Coding and Modification Rules**

* **DO NOT MODIFY** any cell that begins with the comment `### DO NOT MODIFY ###`. These cells contain tests or essential setup.
* You are only allowed to modify cells that begin with the comment `### MODIFY ###`.
* Write all of your code between the designated markers:
    ```python
    ### YOUR CODE STARTS HERE ###
    # ... your solution goes here ...
    ### YOUR CODE ENDS HERE ###
    ```

### **Submission and Grading Policy**

* Not submitting a notebook will result in a **0**.
* Submitting an empty notebook will result in a **0**.
* Submitting a notebook with *any* attempt at a solution (i.e., not empty) guarantees at least **1 point** (assuming no plagiarism).

Please carefully read each cell of the notebook, as they contain guidelines to help you complete the assignments. While you don't have to follow them strictly, we believe that they provide enough help.

Please add your **Name** and **Neptun ID** below!

`Good luck!`

**Name:** Géza Mihályi

**Neptun ID:** IL1VCP

## **0. Necessary Imports**
Import all the necessary packages for this assignment. You will use `nashpy` $\geq 0.0.4$, `numpy` and `matplotlib` for this assignment.

In [1]:
### DO NOT MODIFY ###
!pip -q install nashpy

import warnings
import numpy as np
import itertools as it
import nashpy as nash

warnings.filterwarnings("ignore", category=RuntimeWarning, module="nashpy")
np.set_printoptions(precision=4, suppress=True)
rng = np.random.default_rng(42)
print("Imports OK. Nashpy:", nash.__version__)

Imports OK. Nashpy: 0.0.41



## **Task 1: Pure Nash Equilibria (PNE) (10 points)**

This task requires the implementation of two functions to identify pure best responses (BR) and locate all Pure Nash Equilibria (PNE) in a bimatrix game.

### **Deliverables**

1.  `best_response_masks(A, B)`: Returns two boolean masks (`BR_row`, `BR_col`) marking pure best responses. Ties must be marked.
2.  `find_pne(BR_row, BR_col)`: Returns a list of coordinates for all PNE, derived from the intersection of the two best-response masks.
3.  The solution must successfully pass the **5 assert-based tests provided below**.


In [2]:
### MODIFY ###
def best_response_masks(A: np.ndarray, B: np.ndarray):
    """
    Computes boolean masks marking pure best responses for the Row and Column players.
    Ties must be included.

    Args:
        A (np.ndarray): Row player's payoff matrix (m, n).
        B (np.ndarray): Column player's payoff matrix (m, n).

    Returns:
        tuple[np.ndarray, np.ndarray]: (BR_row, BR_col), both of shape (m, n).
    """
    A = np.asarray(A, dtype=float); B = np.asarray(B, dtype=float)
    assert A.shape == B.shape, "A and B must have same shape"
    ### YOUR CODE STARTS HERE ###
    A_mask = (A == A.max(axis=0, keepdims=True)).astype(int)
    B_mask = (B == B.max(axis=1, keepdims=True)).astype(int)


    BR_row = A_mask
    BR_col = B_mask

    return tuple((BR_row, BR_col))
    ### YOUR CODE ENDS HERE ###

def find_pne(BR_row: np.ndarray, BR_col: np.ndarray):
    """
    Identifies the coordinates of all Pure Nash Equilibria (PNE)
    by finding the intersection of the best-response masks.

    Args:
        BR_row (np.ndarray): Boolean mask for Row player's best responses.
        BR_col (np.ndarray): Boolean mask for Column player's best responses.

    Returns:
        list[list[int]]: A list of [row_index, col_index] lists for all PNE coordinates.
    """
    assert BR_row.shape == BR_col.shape, "Masks must have the same shape"
    ### YOUR CODE STARTS HERE ###

    returnlist = []
    c = np.where(BR_col == BR_row, BR_col, 0)
    if c.size == 0:
        return returnlist
    c = c.reshape(BR_col.shape)
    
    for i in range(c.shape[0]):
        for j in range(c.shape[1]):
            if c[i][j] == 1:
                returnlist.append([i, j])

   

    return returnlist

    raise NotImplementedError
    ### YOUR CODE ENDS HERE ###

## **Task 1: Tests**

In [3]:
### DO NOT MODIFY ###
TOTAL_POINTS = 10
POINTS_PER_TEST = TOTAL_POINTS / 5
score = 0
test_results = []
tests = [
    (
        "Case 1:",
        np.array([[3, 0], [0, 1]]), np.array([[1, 0], [0, 3]]),
        [[0, 0], [1, 1]]
    ),
    (
        "Case 2:",
        np.array([[1, -1], [-1, 1]]), np.array([[-1, 1], [1, -1]]),
        []
    ),
    (
        "Case 3:",
        np.array([[5, 5], [1, 1]]), np.array([[1, 5], [1, 5]]),
        [[0, 1]]
    ),
    (
        "Case 4:",
        np.eye(3), np.eye(3),
        [[0, 0], [1, 1], [2, 2]]
    ),
    (
        "Case 5:",
        np.full((4, 4), 5), np.full((4, 4), 5),
        16 # (number of PNE)
    )
]

print(f"--- Running Task 1 Tests ({POINTS_PER_TEST:.0f} points per test) ---")

for i, (desc, A, B, expected) in enumerate(tests):
    test_id = i + 1
    try:
        br_r, br_c = best_response_masks(A, B)
        pne = find_pne(br_r, br_c)
        is_correct = False
        if test_id == 5:
            is_correct = (len(pne) == expected)
        else:
            is_correct = (pne == expected)

        if is_correct:
            score += POINTS_PER_TEST
            result = "PASSED"
        else:
            result = "FAILED (Incorrect PNE list)"
    except NotImplementedError:
        result = "FAILED (Not Implemented)"
    except Exception as e:
        result = f"ERROR ({type(e).__name__}: {e})"
    test_results.append((test_id, desc, result))
    print(f"{desc} -> {result}")

final_score_1 = score
print("\n---------------------------------------------------")
print(f"Task 1 Grading Summary:")
print(f"Total Tests Passed: {int(score / POINTS_PER_TEST)} / 5")
print(f"Final Score: {score:.1f} / {TOTAL_POINTS:.1f} points")
print("---------------------------------------------------")

--- Running Task 1 Tests (2 points per test) ---
Case 1: -> PASSED
Case 2: -> PASSED
Case 3: -> PASSED
Case 4: -> PASSED
Case 5: -> PASSED

---------------------------------------------------
Task 1 Grading Summary:
Total Tests Passed: 5 / 5
Final Score: 10.0 / 10.0 points
---------------------------------------------------


## **Task 2: Pure Nash Equilibria (PNE) using `nashpy` (5 points)**

This task requires you to use the external library **`nashpy`** to instantiate a game and find all PNE. You must extract the coordinates of the PNE from the library's output and return them in the specified list format. You can find the NashPy documentation [[here]](https://nashpy.readthedocs.io/en/stable/).

### **Deliverables**

1.  `nashpy_find_pne(A, B)`: A function implemented using the `nashpy.Game` object and its enumeration method.
2. The solution **must pass the 5 assert-based tests** provided below.
3. You **must** use the `nashpy.Game` object and its `support_enumeration` method.


In [4]:
### MODIFY ###
def nashpy_find_pne(A: np.ndarray, B: np.ndarray) -> list:
    """
    Finds the coordinates of all Pure Nash Equilibria (PNE) using nashpy.

    Args:
        A (np.ndarray): Row player's payoff matrix.
        B (np.ndarray): Column player's payoff matrix.

    Returns:
        list: A list of PNE coordinates [[r1, c1], [r2, c2], ...].
    """
    assert A.shape == B.shape, "A and B must have same shape"
    ### YOUR CODE STARTS HERE ###
    return_list = []
    game = nash.Game(A, B)
    egs = game.support_enumeration()
    for eq in egs:

        r, c = eq

        row_fully_mixed = np.any((r > 0) & (r < 1))
        col_fully_mixed = np.any((c > 0) & (c < 1))

        if row_fully_mixed and col_fully_mixed:
            continue

        row_idx = int(np.argmax(r))
        col_idx = int(np.argmax(c))
        return_list.append([row_idx, col_idx])
    return return_list
        
    raise NotImplementedError
    ### YOUR CODE ENDS HERE ###

## **Task 2: Tests**

In [5]:
### DO NOT MODIFY ###
TOTAL_POINTS = 5
POINTS_PER_TEST = TOTAL_POINTS / 5
score = 0
test_results = []
tests = [
    (
        "Case 1:",
        np.array([[3, 0], [0, 1]]), np.array([[1, 0], [0, 3]]),
        [[0, 0], [1, 1]]
    ),
    (
        "Case 2:",
        np.array([[1, -1], [-1, 1]]), np.array([[-1, 1], [1, -1]]),
        []
    ),
    (
        "Case 3:",
        np.array([[5, 5], [1, 1]]), np.array([[1, 5], [1, 5]]),
        [[0, 1]]
    ),
    (
        "Case 4:",
        np.eye(3), np.eye(3),
        [[0, 0], [1, 1], [2, 2]]
    ),
    (
        "Case 5:",
        np.full((4, 4), 5), np.full((4, 4), 5),
        16
    )
]

print(f"--- Running Task 2 Tests ({POINTS_PER_TEST:.0f} points per test) ---")
for i, (desc, A, B, expected) in enumerate(tests):
    test_id = i + 1
    try:
        pne = nashpy_find_pne(A, B)
        is_correct = False
        if test_id == 5:
            is_correct = (len(pne) == expected)
        else:
            is_correct = (pne == expected)

        if is_correct:
            score += POINTS_PER_TEST
            result = "PASSED"
        else:
            result = f"FAILED (Incorrect PNE list: Got {pne})"
    except NotImplementedError:
        result = "FAILED (Not Implemented)"
    except Exception as e:
        result = f"ERROR ({type(e).__name__}: {e})"
    print(f"{desc} -> {result}")

final_score_2 = score
print("\n---------------------------------------------------")
print(f"Task 2 Grading Summary:")
print(f"Total Tests Passed: {int(score / POINTS_PER_TEST)} / 5")
print(f"Final Score: {score:.1f} / {TOTAL_POINTS:.1f} points")
print("---------------------------------------------------")

--- Running Task 2 Tests (1 points per test) ---
Case 1: -> PASSED
Case 2: -> PASSED
Case 3: -> PASSED
Case 4: -> PASSED
Case 5: -> PASSED

---------------------------------------------------
Task 2 Grading Summary:
Total Tests Passed: 5 / 5
Final Score: 5.0 / 5.0 points
---------------------------------------------------



## **Task 3: $\varepsilon$-NE gap calculator for mixed profiles (10 points)**

This task requires you to implement a function that calculates the individual **deviation incentive** (or $\varepsilon$-gap) for each player, given a bimatrix game and a pair of mixed strategies $(\mathbf{p}, \mathbf{q})$.

The deviation incentive for a player measures the maximum amount their expected payoff would increase if they unilaterally switched from their current mixed strategy to their **absolute best pure response** against the opponent's strategy.

### **Deliverables**

1.  `epsilon_gaps(A, B, p, q)`: A function that returns the two individual deviation incentives as a tuple of floats: $(\text{gap}_{\text{Row}}, \text{gap}_{\text{Col}})$.
2.  This function will be used later in the notebook to compute the overall **$\varepsilon$-Nash Equilibrium gap** ($\varepsilon = \max(\text{gap}_{\text{Row}}, \text{gap}_{\text{Col}})$) for rounded mixed equilibria.
3.  The solution **must pass the 5 assert-based tests** provided below.

In [6]:
### MODIFY ###
def epsilon_gaps(A: np.ndarray, B: np.ndarray, p: np.ndarray, q: np.ndarray) -> tuple[float, float]:
    """
    Calculates the individual deviation incentives (gaps) for the Row and Column players.

    Args:
        A (np.ndarray): Row player's payoff matrix (m, n).
        B (np.ndarray): Column player's payoff matrix (m, n).
        p (np.ndarray): Row player's mixed strategy vector (m,).
        q (np.ndarray): Column player's mixed strategy vector (n,).

    Returns:
        tuple[float, float]: (Row player gap, Column player gap).
    """
    A = np.asarray(A, dtype=float); B = np.asarray(B, dtype=float)
    p = np.asarray(p, dtype=float); q = np.asarray(q, dtype=float)
    

    assert A.shape == B.shape
    assert np.isclose(p.sum(),1) and np.isclose(q.sum(),1)
    assert (p>=-1e-12).all() and (q>=-1e-12).all()

    ### YOUR CODE STARTS HERE ###

    row_pure_payoffs = A @ q 
    col_pure_payoff = B.T @ p
    row_gap = row_pure_payoffs.max() - row_pure_payoffs @ p
    col_gap = col_pure_payoff.max() - col_pure_payoff @ q
 
    return tuple([row_gap, col_gap])
    raise NotImplementedError
    ### YOUR CODE ENDS HERE ###

## **Task 3: Tests**

In [7]:
### DO NOT MODIFY ###
TOTAL_POINTS = 10
POINTS_PER_TEST = TOTAL_POINTS / 5
score = 0
test_results = []
tests = [
    # Case 1:
    (
        "Case 1:",
        np.array([[1,-1],[-1,1]]), -np.array([[1,-1],[-1,1]]),
        np.array([0.5, 0.5]), np.array([0.5, 0.5]),
        (0.0, 0.0)
    ),

    # Case 2:
    (
        "Case 2:",
        np.array([[3, 0], [5, 1]]), np.array([[3, 5], [0, 1]]),
        np.array([0, 1]), np.array([0, 1]),
        (0.0, 0.0)
    ),

    # Case 3:
    (
        "Case 3:",
        np.array([[3, 0], [0, 1]]), np.array([[1, 0], [0, 3]]),
        np.array([0, 1]), np.array([1, 0]),
        (3.0, 3.0)
    ),

    # Case 4:
    (
        "Case 4:",
        np.array([[3, 0], [0, 1]]), np.array([[1, 0], [0, 3]]),
        np.array([1, 0]), np.array([0, 1]),
        (1.0, 1.0)
    ),

    # Case 5:
    (
        "Case 5:",
        np.array([[2, 0], [0, 1]]), np.array([[1, 0], [0, 2]]),
        np.array([0.5, 0.5]), np.array([0.5, 0.5]),
        (0.25, 0.25)
    )
]

print(f"--- Running Task 3 Tests ({POINTS_PER_TEST:.0f} points per test) ---")

for i, (desc, A, B, p, q, expected) in enumerate(tests):
    test_id = i + 1
    try:
        row_gap, col_gap = epsilon_gaps(A, B, p, q)
        is_row_correct = np.isclose(row_gap, expected[0])
        is_col_correct = np.isclose(col_gap, expected[1])

        if is_row_correct and is_col_correct:
            score += POINTS_PER_TEST
            result = "PASSED"
        else:
            result = f"FAILED (Got ({row_gap:.2f}, {col_gap:.2f}), Expected {expected})"
    except NotImplementedError:
        result = "FAILED (Not Implemented)"
    except Exception as e:
        result = f"ERROR ({type(e).__name__}: {e})"
    print(f"{desc} -> {result}")

final_score_3 = score
print("\n---------------------------------------------------")
print(f"Task 3 Grading Summary:")
print(f"Total Tests Passed: {int(score / POINTS_PER_TEST)} / 5")
print(f"Final Score: {score:.1f} / {TOTAL_POINTS:.1f} points")
print("---------------------------------------------------")

--- Running Task 3 Tests (2 points per test) ---
Case 1: -> PASSED
Case 2: -> PASSED
Case 3: -> PASSED
Case 4: -> PASSED
Case 5: -> PASSED

---------------------------------------------------
Task 3 Grading Summary:
Total Tests Passed: 5 / 5
Final Score: 10.0 / 10.0 points
---------------------------------------------------


## **Task 4: Statistical Analysis of Random Games (10 points)**

This task requires you to generate a large batch of random games and statistically analyze their structural properties, specifically the frequency of **PNE**.

### **Deliverables**

1. `analyze_random_games(size, num_games, seed)`: A function that generates and analyzes a batch of random games, returning the frequency of PNE.
2. The function must pass a small, seeded test case to verify correctness and reproducibility.
3. Generate $(\text{size} \times \text{size})$ matrices where payoffs are drawn from a uniform integer distribution (e.g., 0 to 100).
4. You **must** use the provided `seed` to set the NumPy random state using `np.random.seed()`.
5. You must utilize your previously defined PNE-finding logic (`best_response_masks` and `find_pne`).
6. The function must return the percentage (float, 0.0 to 100.0) of games that contain at least one PNE.


In [8]:
### MODIFY ###
def analyze_random_games(size: int, num_games: int, seed: int) -> float:
    """
    Generates a batch of random bimatrix games (of size x size) and computes
    the statistical frequency of games containing at least one Pure Nash Equilibrium (PNE).

    Args:
        size (int): The dimension of the square payoff matrices
        num_games (int): The total number of independent random games to generate and analyze.
        seed (int): The NumPy random seed

    Returns:
        float: The percentage of the generated games (0.0 to 100.0) that contain
               one or more Pure Nash Equilibria.
    """
    np.random.seed(seed)
    games_with_pne = 0
    ### YOUR CODE STARTS HERE ###

    for i in range(num_games):
        A = np.random.randint(0, 101, (size, size))
        B = np.random.randint(0, 101, (size, size))


        br_r, br_c = best_response_masks(A, B)
        pne = find_pne(br_r, br_c)
        if len(pne) >= 1:
            games_with_pne += 1

    return round((games_with_pne/num_games)*100, 1)

    raise NotImplementedError
    ### YOUR CODE ENDS HERE ###

## **Task 4: Tests**

In [9]:
### DO NOT MODIFY ###
TOTAL_POINTS = 10
POINTS_PER_TEST = TOTAL_POINTS / 2
score = 0
num_tests = 2
passed_tests = 0

SEED_1 = 42
SIZE_1 = 3
NUM_GAMES_1 = 100
EXPECTED_FREQ_1 = 78.0

SEED_2 = 101
SIZE_2 = 4
NUM_GAMES_2 = 100
EXPECTED_FREQ_2 = 76.0

tests = [
    (1, SIZE_1, NUM_GAMES_1, SEED_1, EXPECTED_FREQ_1),
    (2, SIZE_2, NUM_GAMES_2, SEED_2, EXPECTED_FREQ_2)
]
print(f"--- Running Task 4 Tests ({POINTS_PER_TEST:.0f} points per test) ---")
for test_id, size, num_games, seed, expected_freq in tests:
    try:
        pne_frequency = analyze_random_games(size, num_games, seed)
        if np.isclose(pne_frequency, expected_freq, atol=1.0, rtol=0.0):
            score += POINTS_PER_TEST
            passed_tests += 1
            result = f"PASSED (Got {pne_frequency:.1f}%)"
        else:
            result = f"FAILED (Got {pne_frequency:.1f}%, Expected {expected_freq:.1f}% for {size}x{size} game, seed {seed})"
    except NotImplementedError:
        result = "FAILED (Not Implemented)"
    except Exception as e:
        result = f"ERROR ({type(e).__name__}: {e})"
    print(f"Case {test_id}: {size}x{size} -> {result}")

final_score_4 = score
print("\n---------------------------------------------------")
print(f"Task 4 Grading Summary:")
print(f"Total Tests Passed: {passed_tests} / {num_tests}")
print(f"Final Score: {score:.1f} / {TOTAL_POINTS:.1f} points")
print("---------------------------------------------------")

--- Running Task 4 Tests (5 points per test) ---
Case 1: 3x3 -> PASSED (Got 78.0%)
Case 2: 4x4 -> PASSED (Got 77.0%)

---------------------------------------------------
Task 4 Grading Summary:
Total Tests Passed: 2 / 2
Final Score: 10.0 / 10.0 points
---------------------------------------------------


## **Task 5: Identifying Strictly Dominated Strategies (10 points)**

This exercise requires you to analyze a two-player, simultaneous-move game and identify all pure strategies that are **strictly dominated** by another pure strategy. A strategy $s$ is **strictly dominated** if there exists an alternative strategy $s'$ that yields a strictly higher payoff for the player, regardless of the opponent's action.

### **Deliverables**

1. Implement the function `find_strictly_dominated` which analyzes the payoff matrices $\mathbf{A}$ (Row) and $\mathbf{B}$ (Column) and returns the indices of all strictly dominated pure strategies for both players.
2. The function must return a tuple containing two lists of integer indices: `(dominated_row_indices, dominated_col_indices)`.
3. The lists must be returned in **sorted ascending order**.

In [10]:
### MODIFY ###
def find_strictly_dominated(A: np.ndarray, B: np.ndarray) -> tuple[list[int], list[int]]:
    """
    Identifies all pure strategies for the Row and Column players that are
    strictly dominated by another pure strategy.

    Args:
        A (np.ndarray): The payoff matrix for the Row player.
        B (np.ndarray): The payoff matrix for the Column player.
                        A.shape must equal B.shape.

    Returns:
        tuple[list[int], list[int]]: A tuple containing two sorted lists of indices:
                                     1. Indices of Row's strictly dominated strategies.
                                     2. Indices of Column's strictly dominated strategies.
    """
    assert A.shape == B.shape
    R, C = A.shape
    dominated_rows = []
    dominated_cols = []



    ### YOUR CODE STARTS HERE ###
    for i in range(R):
        mask = np.ones(A.shape, bool)
        mask[i] = False
        remaining = A[mask]
        remaining = remaining.reshape((-1,C))
        dominated_matrix = A[i] > remaining
        cheching_row = np.full((1, C), False)
        dominated_matrix = np.insert(dominated_matrix, i, cheching_row, axis=0)
        for i in range(len(dominated_matrix)):
            if np.all(dominated_matrix[i] == True):
                dominated_rows.append(i)


    B = B.T

    for i in range(C):
        mask = np.ones(B.shape, bool)
        mask[i] = False
        remaining = B[mask]
        remaining = remaining.reshape((-1,R))
        dominated_matrix = B[i] > remaining
        cheching_row = np.full((1, R), False)
        dominated_matrix = np.insert(dominated_matrix, i, cheching_row, axis=0)
        for i in range(len(dominated_matrix)):
            if np.all(dominated_matrix[i] == True):
                dominated_cols.append(i)

    dominated_rows = list(set(dominated_rows))
    dominated_cols = list(set(dominated_cols))

    dominated_rows.sort()
    dominated_cols.sort()

    return tuple([dominated_rows, dominated_cols])

    raise NotImplementedError
    ### YOUR CODE ENDS HERE ###

## **Task 5: Tests**

In [11]:
### DO NOT MODIFY ###
TOTAL_POINTS = 10
POINTS_PER_TEST = TOTAL_POINTS / 5
score = 0
test_results = []
tests = [
    (
        "Case 1:",
        np.array([[3, 0], [5, 1]]), np.array([[3, 5], [0, 1]]),
        [0], [0]
    ),
    (
        "Case 2:",
        np.array([[5, 7, 7], [4, 4, 4], [1, 6, 3]]), np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]),
        [1, 2], []
    ),
    (
        "Case 3:",
        np.array([[1, 3], [3, 1]]), np.array([[1, 1], [1, 1]]),
        [], []
    ),
    (
        "Case 4:",
        np.array([[2, 1, 0], [0, 1, 2]]), np.array([[3, 1, 0], [3, 4, 0]]),
        [], [2]
    ),
    (
        "Case 5:",
        np.array([[5, 0], [6, 1]]), np.array([[1, 0], [1, 0]]),
        [0], [1]
    )
]

print(f"--- Running Task 5 Tests ({POINTS_PER_TEST:.0f} points per test) ---")
for i, (desc, A, B, r_expected, c_expected) in enumerate(tests):
    test_id = i + 1
    try:
        r_got, c_got = find_strictly_dominated(A, B)
        r_match = r_got == r_expected
        c_match = c_got == c_expected
        is_correct = r_match and c_match
        if is_correct:
            score += POINTS_PER_TEST
            result_str = "PASSED"
        else:
            result_str = "FAILED"
            error_message = f" (Got R:{r_got}, C:{c_got} | Expected R:{r_expected}, C:{c_expected})"
        print(f"{desc} -> {result_str}{error_message if not is_correct else ''}")
    except NotImplementedError:
        print(f"{desc} -> FAILED (Not Implemented)")
    except Exception as e:
        print(f"{desc} -> ERROR ({type(e).__name__}: {e})")
total_tests_passed = int(score / POINTS_PER_TEST)

final_score_5 = score
print("\n---------------------------------------------------")
print(f"Task 5 Grading Summary:")
print(f"Total Tests Passed: {total_tests_passed} / {len(tests)}")
print(f"Final Score: {score:.1f} / {TOTAL_POINTS:.1f} points")
print("---------------------------------------------------")

--- Running Task 5 Tests (2 points per test) ---
Case 1: -> PASSED
Case 2: -> PASSED
Case 3: -> PASSED
Case 4: -> PASSED
Case 5: -> PASSED

---------------------------------------------------
Task 5 Grading Summary:
Total Tests Passed: 5 / 5
Final Score: 10.0 / 10.0 points
---------------------------------------------------


## **Task 6: Axelrod Tournament Simulation (5 points)**

This exercise requires you to simulate a simplified, round-robin Axelrod Tournament using the famous **Iterated Prisoner's Dilemma (IPD)** game. You will implement a few common IPD strategies and calculate their total scores when pitted against each other.

### **Payoff Matrix**

The game is the standard Prisoner's Dilemma (PD), played repeatedly. `C=0` and `D=1`.

| | **C** | **D** |
| :---: | :---: | :---: |
| **C** | (3, 3) | (0, 5) |
| **D** | (5, 0) | (1, 1) |

### **Strategies**

Implement the following three strategies as Python functions. Each function takes the history of moves (your own and your opponent's) and returns the next move (0 or 1).

* **`TitForTat`**
* **`AlwaysDefect`**
* **`Random`**

### **Deliverables**

Implement the `run_tournament` function to simulate the competition.

1.  `run_tournament(strategies, rounds, seed)`: A function that simulates the round-robin tournament where every strategy plays every other strategy **including itself**. Make sure to incorporate the already given `_play_match(p1_strategy, p2_strategy, rounds)` function.
2.  The simulation must use the provided `seed` via `np.random.seed(seed)` to ensure the `Random` strategy is reproducible.
3.  The function must return a **dictionary** mapping the strategy name (string) to its **total cumulative score** across all matches.


In [12]:
### MODIFY ###
def tit_for_tat(history_self: np.ndarray, history_opponent: np.ndarray) -> int:
    """Cooperate on the first move, then copy opponent's last move."""
    ### YOUR CODE STARTS HERE ###
    if len(history_self)==0:
        return 0
    else:
        return history_opponent[-1]
    raise NotImplementedError
    ### YOUR CODE ENDS HERE ###

def always_defect(history_self: np.ndarray, history_opponent: np.ndarray) -> int:
    """Always chooses Defect."""
    ### YOUR CODE STARTS HERE ###
    return 1
    raise NotImplementedError
    ### YOUR CODE ENDS HERE ###

def random(history_self: np.ndarray, history_opponent: np.ndarray) -> int:
    """Chooses Cooperate or Defect with 50% probability."""
    ### YOUR CODE STARTS HERE ###
    return_value = np.random.randint(0, 2)
    return return_value

    raise NotImplementedError
    ### YOUR CODE ENDS HERE ###

def run_tournament(strategies: dict, rounds: int, seed: int) -> dict:
    """
    Simulates a round-robin Axelrod tournament for the Iterated Prisoner's Dilemma.

    Args:
        strategies (dict): A dict mapping strategy names (str) to their function objects.
        rounds (int): The number of rounds each pair-wise match should run.
        seed (int): The seed for the random number generator.

    Returns:
        dict: A dictionary mapping strategy names (str) to their total scores (int).
    """
    np.random.seed(seed)

    strategy_names = list(strategies.keys())
    scores = {name: 0 for name in strategy_names}
    ### YOUR CODE STARTS HERE ###
    
    already_played = []

    for strat1 in strategy_names:
        for strat2 in strategy_names:
            if (strat2, strat1) in already_played:
                continue
            scorestrat1, scorestart2 = _play_match(strategies[strat1], strategies[strat2], rounds)
            
            if strat1 == strat2:
                scores[strat1] += scorestrat1
            else:
                scores[strat1] += scorestrat1
                scores[strat2] += scorestart2
            already_played.append((strat1, strat2))

    return scores

    raise NotImplementedError
    ### YOUR CODE ENDS HERE ###

# Payoffs defined globally for efficiency: PAYOFFS[Your Move][Opponent's Move] -> Your Payoff
# C=0, D=1
PAYOFFS = np.array([[3, 0], [5, 1]])

def _play_match(p1_strategy, p2_strategy, rounds) -> tuple[int, int]:
    """
    Simulates a single, fixed-length match of the Iterated Prisoner's Dilemma
    between two competing strategies.

    This function executes the core game loop, handling:
    1. Strategy callback: Calling each strategy function with the current history.
    2. Score aggregation: Calculating scores based on the global PAYOFFS matrix.
    3. History maintenance: Appending the current moves to the respective histories.

    Args:
        p1_strategy (callable): The function implementing Player 1's logic.
        p2_strategy (callable): The function implementing Player 2's logic.
        rounds (int): The number of rounds the match should run for.

    Returns:
        tuple[int, int]: The total cumulative scores for Player 1 and Player 2, respectively.
    """
    history_p1 = np.array([], dtype=int)
    history_p2 = np.array([], dtype=int)
    score_p1 = 0
    score_p2 = 0

    for _ in range(rounds):

        m1 = p1_strategy(history_p1, history_p2)
        m2 = p2_strategy(history_p2, history_p1)

        score_p1 += PAYOFFS[m1][m2]
        score_p2 += PAYOFFS[m2][m1]

        history_p1 = np.append(history_p1, m1)
        history_p2 = np.append(history_p2, m2)

    return score_p1, score_p2

## **Task 6: Tests**

In [13]:
### DO NOT MODIFY ###
TOTAL_POINTS = 5
POINTS_PER_TEST = TOTAL_POINTS / 2
score = 0
total_tests_passed = 0
ALL_STRATEGIES = {
    "TFT": tit_for_tat,
    "AllD": always_defect,
    "Rndm": random
}

def check_match_set(rounds, seed, expected_scores):
    TFT = tit_for_tat
    AllD = always_defect
    Rndm = random

    s1, s2 = _play_match(TFT, AllD, rounds)

    np.random.seed(seed)
    s3, s4 = _play_match(TFT, Rndm, rounds)

    np.random.seed(seed)
    s5, s6 = _play_match(AllD, Rndm, rounds)
    

    actual_scores = (s1, s2, s3, s4, s5, s6)
    return actual_scores == expected_scores, actual_scores

tests = [
    (
        "Case 1:",
        3, 42,
        (2, 7, 8, 8, 11, 1),
        "match_test"
    ),
    (
        "Case 2:",
        3, 42,
        {"TFT": 19, "AllD": 21, "Rndm": 15},
        "tournament_test"
    )
]
print(f"--- Running Task 6 Tests ({POINTS_PER_TEST:.1f} points per test) ---")
for i, test_data in enumerate(tests):
    desc, rounds, seed, expected, test_type = test_data
    error_message = ""
    is_correct = False
    try:
        if test_type == "match_test":
            is_correct, actual_scores = check_match_set(rounds, seed, expected)
            r_expected = f"({expected[0]}, {expected[1]}) ({expected[2]}, {expected[3]}) ({expected[4]}, {expected[5]})"
            r_got = f"({actual_scores[0]}, {actual_scores[1]}) ({actual_scores[2]}, {actual_scores[3]}) ({actual_scores[4]}, {actual_scores[5]})"
            if not is_correct:
                 error_message = f" (Got: {r_got} | Expected: {r_expected})"
        elif test_type == "tournament_test":
            r_got = run_tournament(ALL_STRATEGIES, rounds, seed)
            r_expected = expected
            is_correct = (r_got == r_expected)
            if not is_correct:
                error_message = f" (Got: {r_got} | Expected: {r_expected})"
        if is_correct:
            score += POINTS_PER_TEST
            total_tests_passed += 1
            result_str = "PASSED"
        else:
            result_str = "FAILED"
        print(f"{desc} -> {result_str}{error_message}")
    except NotImplementedError:
        print(f"{desc} -> FAILED (Not Implemented)")
    except Exception as e:
        print(f"{desc} -> ERROR ({type(e).__name__}: {e})")

final_score_6 = score
print("\n---------------------------------------------------")
print(f"Task 6 Grading Summary (Axelrod Tournament):")
print(f"Total Tests Passed: {total_tests_passed} / {len(tests)}")
print(f"Final Score: {score:.1f} / {TOTAL_POINTS:.1f} points")
print("---------------------------------------------------")

--- Running Task 6 Tests (2.5 points per test) ---
Case 1: -> PASSED
Case 2: -> PASSED

---------------------------------------------------
Task 6 Grading Summary (Axelrod Tournament):
Total Tests Passed: 2 / 2
Final Score: 5.0 / 5.0 points
---------------------------------------------------


# **Total Score**

In [14]:
### DO NOT MODIFY ###
total_score = 0
task_scores = {}
for i in range(1, 7):
    score_var_name = f'final_score_{i}'
    if score_var_name in locals():
        task_scores[f'Task {i}'] = locals()[score_var_name]
        total_score += locals()[score_var_name]
    else:
        task_scores[f'Task {i}'] = 0
        print(f"Warning: {score_var_name} not found. Ensure all test cells are run.")
for task, score in task_scores.items():
    print(f"{task} Score: {score:.1f} points")
print("---------------------------------")
print(f"Total Score: {total_score:.1f}/100.0")
print("---------------------------------")

Task 1 Score: 10.0 points
Task 2 Score: 5.0 points
Task 3 Score: 10.0 points
Task 4 Score: 10.0 points
Task 5 Score: 10.0 points
Task 6 Score: 5.0 points
---------------------------------
Total Score: 50.0/100.0
---------------------------------


Please make sure to download your `.ipynb` file, and upload it to **Canvas** on time!

<img src="https://www.usatoday.com/gcdn/authoring/authoring-images/2023/08/25/USAT/70680172007-alertsm.png?crop=2099,1187,x0,y156" alt="1" border="0">



In [15]:
# @title ⏰ Time Left Until Submission ⏰
# %%capture flowchart_output
# HIDDEN CELL

from datetime import datetime, timedelta, UTC

deadline = datetime(2025, 11, 5, 23, 59, 0, tzinfo=UTC)

def time_until_deadline():
    now = datetime.now(UTC)
    remaining = deadline - now
    if remaining.total_seconds() <= 0:
        return "Time's up!"
    days = remaining.days
    hours, remainder = divmod(remaining.seconds, 3600)
    minutes, _ = divmod(remainder, 60)

    return f"{days} days, {hours} hours, {minutes} minutes"

print("Time left until submission:", time_until_deadline())

Time left until submission: 25 days, 3 hours, 2 minutes
